<a href="https://colab.research.google.com/github/benza613/Sentiment-Analysis-using-Deep-Learning/blob/master/notebooks/GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
%cd gdrive/My\ Drive/SNLP

In [0]:
%cd Sentiment-Analysis-using-Deep-Learning/

In [0]:
#used to unzip the glove zip folder
#import zipfile
#with zipfile.ZipFile('glove.840B.300d.zip', 'r') as zip_ref:
 #   zip_ref.extractall('glove.840B.300d')

In [0]:
Train_ZipCSVFileName = 'data/Train_2000_Apr-03-2020_06-46.csv'
Test_ZipCSVFileName = 'data/Test_2000_Apr-03-2020_06-46.csv'

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()

In [0]:
# Get names of indexes for which column overall has value 3
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 1
df_train.loc[(df_train.overall == 2),'overall']= 1
df_train.loc[(df_train.overall == 4),'overall']= 5
df_train.loc[(df_train.overall == 5),'overall']= 5

df_test.loc[(df_test.overall == 1),'overall']= 1
df_test.loc[(df_test.overall == 2),'overall']= 1
df_test.loc[(df_test.overall == 4),'overall']= 5
df_test.loc[(df_test.overall == 5),'overall']= 5

df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 

In [0]:
df_train['overall'].describe()
df_test['overall'].describe()

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200

import uuid
folderGUID = uuid.uuid4().hex
#folderGUID="b3133d6101874b4ba9158c9e261b2daf"
# stupid shell way of converting variable to string 
print(folderGUID)

!mkdir "ModelResults/$folderGUID"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [0]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['summary'] + ' DELIM '+df_train['reviewText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

In [0]:
X = tokenizer.texts_to_sequences(df_train['summary'].values + ' DELIM '+df_train['reviewText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


In [0]:
Y = pd.get_dummies(df_train['overall']).values
print('Shape of label tensor:', Y.shape)

In [0]:
embedding_index = {}

f = open('glove.6B/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs
f.close()

print('found word vecs: ',len(embedding_index))
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index)+1,embedding_dim))
embedding_matrix.shape
for word,i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

#model
from keras.layers import Embedding
from keras.layers import Bidirectional,GlobalMaxPool1D,Conv1D
from keras.layers import LSTM,Input,Dense,Dropout,Activation
from keras.models import Model


Below are the functions for recall, F1 and precissio

In [0]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
embedding_layer = Embedding(len(word_index)+1,embedding_dim,weights=[embedding_matrix],input_length=MAX_SEQUENCE_LENGTH,trainable=False)
inp = Input(shape=(MAX_SEQUENCE_LENGTH,))
x = embedding_layer(inp)
x = Bidirectional(LSTM(50,return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50,activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(2,activation='sigmoid')(x)
model = Model(inputs=inp,outputs=x)

#call the functions in the metrics 
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc',f1_m,precision_m, recall_m])

In [0]:
model.fit(X,Y,validation_split=0.3, epochs=2, verbose=0);


X_test = tokenizer.texts_to_sequences(df_test['summary'].values + ' DELIM '+df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, Y_Test, verbose=0)

The values are printed

In [0]:
print("loss",loss)
print("accuracy",accuracy)
print("f1 score",f1_score)
print("precision",precision)
print("recall",recall)

In [0]:
filePath = "ModelResults/"+str(folderGUID)+"/model.h5"
model.save(filePath)
print("Saved model to disk : "+ str(folderGUID))

In [0]:
X_test = tokenizer.texts_to_sequences(df_test['summary'].values + ' DELIM '+df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

accr = model.evaluate(X_test,Y_Test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [0]:
# Test for single review 
new_review = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_review)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)

pred = model.predict(padded)

print(pred)
print(np.argmax(pred))